In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, TimestampType
from datetime import datetime

data = [
    (datetime(2020, 1, 15).date(), datetime(2020, 1, 15, 10, 30, 0)),
    (datetime(2021, 5, 20).date(), datetime(2021, 5, 20, 14, 45, 0)),
    (datetime(2022, 8, 10).date(), datetime(2022, 8, 10, 9, 15, 0))
]

schema = StructType([
    StructField("date", DateType(), True),
    StructField("timestamp", TimestampType(), True)
])

df = spark.createDataFrame(data, schema)

df.show(truncate=False)


+----------+-------------------+
|date      |timestamp          |
+----------+-------------------+
|2020-01-15|2020-01-15 10:30:00|
|2021-05-20|2021-05-20 14:45:00|
|2022-08-10|2022-08-10 09:15:00|
+----------+-------------------+



In [0]:
from pyspark.sql.functions import *

ts_toSecond = df.withColumn("unix_col", unix_timestamp(df.timestamp))

display(ts_toSecond)

date,timestamp,unix_col
2020-01-15,2020-01-15T10:30:00.000+0000,1579084200
2021-05-20,2021-05-20T14:45:00.000+0000,1621521900
2022-08-10,2022-08-10T09:15:00.000+0000,1660122900


In [0]:
s_toDate = ts_toSecond.withColumn("secondtoTimestamp", from_unixtime("unix_col"))

display(s_toDate)

date,timestamp,unix_col,secondtoTimestamp
2020-01-15,2020-01-15T10:30:00.000+0000,1579084200,2020-01-15 10:30:00
2021-05-20,2021-05-20T14:45:00.000+0000,1621521900,2021-05-20 14:45:00
2022-08-10,2022-08-10T09:15:00.000+0000,1660122900,2022-08-10 09:15:00


In [0]:
timestamp_toDate = df.withColumn("ts_todate", to_date("timestamp"))

display(timestamp_toDate)

date,timestamp,ts_todate
2020-01-15,2020-01-15T10:30:00.000+0000,2020-01-15
2021-05-20,2021-05-20T14:45:00.000+0000,2021-05-20
2022-08-10,2022-08-10T09:15:00.000+0000,2022-08-10


In [0]:
minutes = df.withColumn("minutes", minute("timestamp"))

display(minutes)

date,timestamp,minutes
2020-01-15,2020-01-15T10:30:00.000+0000,30
2021-05-20,2021-05-20T14:45:00.000+0000,45
2022-08-10,2022-08-10T09:15:00.000+0000,15


In [0]:
seconds2 = df.withColumn("seconds", second("timestamp"))
display(seconds2)

date,timestamp,seconds
2020-01-15,2020-01-15T10:30:00.000+0000,0
2021-05-20,2021-05-20T14:45:00.000+0000,0
2022-08-10,2022-08-10T09:15:00.000+0000,0


In [0]:
df.collect()

Out[222]: [Row(date=datetime.date(2020, 1, 15), timestamp=datetime.datetime(2020, 1, 15, 10, 30)),
 Row(date=datetime.date(2021, 5, 20), timestamp=datetime.datetime(2021, 5, 20, 14, 45)),
 Row(date=datetime.date(2022, 8, 10), timestamp=datetime.datetime(2022, 8, 10, 9, 15))]

In [0]:
read_csv = spark.read.csv("dbfs:/FileStore/tables/sample_dataset.csv", header = "true", inferSchema = "true")

display(read_csv)

Department,Employee,Salary,Age,Joining_Date
HR,John,50000,28,2020-01-15
HR,Jane,60000,34,2019-03-23
IT,Mike,75000,29,2021-06-01
IT,Anna,80000,31,2018-07-19
Finance,Sara,65000,45,2017-11-30
Finance,Tom,70000,41,2016-09-15
Marketing,Nina,55000,26,2022-02-10
Marketing,Alex,72000,30,2019-12-05


In [0]:
from pyspark.sql.functions import substring

In [0]:

col = read_csv.withColumn("substring", substring("Department", 1,3))

display(col)

Department,Employee,Salary,Age,Joining_Date,substring
HR,John,50000,28,2020-01-15,HR
HR,Jane,60000,34,2019-03-23,HR
IT,Mike,75000,29,2021-06-01,IT
IT,Anna,80000,31,2018-07-19,IT
Finance,Sara,65000,45,2017-11-30,Fin
Finance,Tom,70000,41,2016-09-15,Fin
Marketing,Nina,55000,26,2022-02-10,Mar
Marketing,Alex,72000,30,2019-12-05,Mar


In [0]:
from pyspark.sql.functions import *
addition = read_csv.select("Department", (col("Salary") + 1).alias("addedsalary"))

display(addition)

Department,addedsalary
HR,50001
HR,60001
IT,75001
IT,80001
Finance,65001
Finance,70001
Marketing,55001
Marketing,72001


In [0]:
lit = read_csv.withColumn("Grade", lit("1.6"))

display(lit)

Department,Employee,Salary,Age,Joining_Date,Grade
HR,John,50000,28,2020-01-15,1.6
HR,Jane,60000,34,2019-03-23,1.6
IT,Mike,75000,29,2021-06-01,1.6
IT,Anna,80000,31,2018-07-19,1.6
Finance,Sara,65000,45,2017-11-30,1.6
Finance,Tom,70000,41,2016-09-15,1.6
Marketing,Nina,55000,26,2022-02-10,1.6
Marketing,Alex,72000,30,2019-12-05,1.6


In [0]:
when1 = read_csv.select("Department", when(col("Salary") > 60000,"true"))
display(when1)

Department,CASE WHEN (Salary > 60000) THEN true END
HR,null
HR,null
IT,true
IT,true
Finance,true
Finance,true
Marketing,null
Marketing,true


In [0]:

agg = read_csv.groupBy("Department")\
.agg(count("Salary").alias("Salary_count"),
     max("Salary").alias("max_salary"),
     min("Salary").alias("min_salary"),
     sum("Salary").alias("sum_salary"),
     avg("Salary").alias("avg_salary"),
     mean("Salary").alias("mean_Salary"),
     mode("Salary").alias("mode_salary") ##value that appears most frequently or smallest
     )

display(agg)

Department,Salary_count,max_salary,min_salary,sum_salary,avg_salary,mean_Salary,mode_salary
HR,2,60000,50000,110000,55000.0,55000.0,50000
Finance,2,70000,65000,135000,67500.0,67500.0,70000
Marketing,2,72000,55000,127000,63500.0,63500.0,55000
IT,2,80000,75000,155000,77500.0,77500.0,75000


In [0]:
round1 = lit.withColumn("rounded", round(col("Grade")))
display(round1)

Department,Employee,Salary,Age,Joining_Date,Grade,rounded
HR,John,50000,28,2020-01-15,1.6,2.0
HR,Jane,60000,34,2019-03-23,1.6,2.0
IT,Mike,75000,29,2021-06-01,1.6,2.0
IT,Anna,80000,31,2018-07-19,1.6,2.0
Finance,Sara,65000,45,2017-11-30,1.6,2.0
Finance,Tom,70000,41,2016-09-15,1.6,2.0
Marketing,Nina,55000,26,2022-02-10,1.6,2.0
Marketing,Alex,72000,30,2019-12-05,1.6,2.0


In [0]:
subtraction = lit.withColumn("sub", col("Age") - col("Salary"))

display(subtraction)

Department,Employee,Salary,Age,Joining_Date,Grade,sub
HR,John,50000,28,2020-01-15,1.6,-49972
HR,Jane,60000,34,2019-03-23,1.6,-59966
IT,Mike,75000,29,2021-06-01,1.6,-74971
IT,Anna,80000,31,2018-07-19,1.6,-79969
Finance,Sara,65000,45,2017-11-30,1.6,-64955
Finance,Tom,70000,41,2016-09-15,1.6,-69959
Marketing,Nina,55000,26,2022-02-10,1.6,-54974
Marketing,Alex,72000,30,2019-12-05,1.6,-71970


In [0]:
abs1 = subtraction.withColumn('abs', abs(col("sub").alias("abs")))

display(abs1)

Department,Employee,Salary,Age,Joining_Date,Grade,sub,abs
HR,John,50000,28,2020-01-15,1.6,-49972,49972
HR,Jane,60000,34,2019-03-23,1.6,-59966,59966
IT,Mike,75000,29,2021-06-01,1.6,-74971,74971
IT,Anna,80000,31,2018-07-19,1.6,-79969,79969
Finance,Sara,65000,45,2017-11-30,1.6,-64955,64955
Finance,Tom,70000,41,2016-09-15,1.6,-69959,69959
Marketing,Nina,55000,26,2022-02-10,1.6,-54974,54974
Marketing,Alex,72000,30,2019-12-05,1.6,-71970,71970


In [0]:
sqrt = lit.withColumn("sqrt", sqrt(col("age")))

display(sqrt)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt
HR,John,50000,28,2020-01-15,1.6,5.291502622129181
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661


In [0]:
display(sqrt)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt
HR,John,50000,28,2020-01-15,1.6,5.291502622129181
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661


In [0]:
from pyspark.sql.functions import concat


In [0]:
concatination = sqrt.withColumn("concat", concat(col("Employee"), col("Age")))

display(concatination)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt,concat
HR,John,50000,28,2020-01-15,1.6,5.291502622129181,John28
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301,Jane34
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504,Mike29
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215,Anna31
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369,Sara45
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485,Tom41
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845,Nina26
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661,Alex30


In [0]:
concat_ws1 = sqrt.withColumn("concatws", concat_ws("/", col("Age"),col("Employee")))

display(concat_ws1)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt,concatws
HR,John,50000,28,2020-01-15,1.6,5.291502622129181,28/John
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301,34/Jane
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504,29/Mike
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215,31/Anna
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369,45/Sara
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485,41/Tom
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845,26/Nina
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661,30/Alex


In [0]:
display(sqrt)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt
HR,John,50000,28,2020-01-15,1.6,5.291502622129181
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661


In [0]:

from pyspark.sql.functions import col, length

df_with_length = sqrt.withColumn("Name_Length", length(col("Employee")))
df_with_length.show()


+----------+--------+------+---+------------+-----+------------------+-----------+
|Department|Employee|Salary|Age|Joining_Date|Grade|              sqrt|Name_Length|
+----------+--------+------+---+------------+-----+------------------+-----------+
|        HR|    John| 50000| 28|  2020-01-15|  1.6| 5.291502622129181|          4|
|        HR|    Jane| 60000| 34|  2019-03-23|  1.6| 5.830951894845301|          4|
|        IT|    Mike| 75000| 29|  2021-06-01|  1.6| 5.385164807134504|          4|
|        IT|    Anna| 80000| 31|  2018-07-19|  1.6|5.5677643628300215|          4|
|   Finance|    Sara| 65000| 45|  2017-11-30|  1.6| 6.708203932499369|          4|
|   Finance|     Tom| 70000| 41|  2016-09-15|  1.6|6.4031242374328485|          3|
| Marketing|    Nina| 55000| 26|  2022-02-10|  1.6|5.0990195135927845|          4|
| Marketing|    Alex| 72000| 30|  2019-12-05|  1.6| 5.477225575051661|          4|
+----------+--------+------+---+------------+-----+------------------+-----------+



In [0]:
from pyspark.sql.functions import upper, lower

upper2 = sqrt.withColumn("Upper", upper(col("Employee")))\
             .withColumn("Lower", lower(col("Employee")))

display(upper2)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt,Upper,Lower
HR,John,50000,28,2020-01-15,1.6,5.291502622129181,JOHN,john
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301,JANE,jane
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504,MIKE,mike
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215,ANNA,anna
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369,SARA,sara
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485,TOM,tom
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845,NINA,nina
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661,ALEX,alex


In [0]:
from pyspark.sql.functions import split

splitting = sqrt.withColumn("FirstName", split(col("joining_date"), "-"))


display(splitting)

Department,Employee,Salary,Age,Joining_Date,Grade,sqrt,FirstName
HR,John,50000,28,2020-01-15,1.6,5.291502622129181,"List(2020, 01, 15)"
HR,Jane,60000,34,2019-03-23,1.6,5.830951894845301,"List(2019, 03, 23)"
IT,Mike,75000,29,2021-06-01,1.6,5.385164807134504,"List(2021, 06, 01)"
IT,Anna,80000,31,2018-07-19,1.6,5.5677643628300215,"List(2018, 07, 19)"
Finance,Sara,65000,45,2017-11-30,1.6,6.708203932499369,"List(2017, 11, 30)"
Finance,Tom,70000,41,2016-09-15,1.6,6.4031242374328485,"List(2016, 09, 15)"
Marketing,Nina,55000,26,2022-02-10,1.6,5.0990195135927845,"List(2022, 02, 10)"
Marketing,Alex,72000,30,2019-12-05,1.6,5.477225575051661,"List(2019, 12, 05)"


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, ArrayType, StringType

# Create Spark session
spark = SparkSession.builder.appName("ArrayColumnExample").getOrCreate()

# Corrected data format
data = [
    (["Alice", "Johnson"],),
    (["Bob", "Smith"],),
    (["Charlie", "Brown"],)
]

# Define schema
schema = StructType([
    StructField("NameParts", ArrayType(StringType()), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

df.show(truncate=False)


+----------------+
|NameParts       |
+----------------+
|[Alice, Johnson]|
|[Bob, Smith]    |
|[Charlie, Brown]|
+----------------+



In [0]:
from pyspark.sql.functions import size
size = df.withColumn("size", size(col("NameParts")))
display(size)

NameParts,size
"List(Alice, Johnson)",2
"List(Bob, Smith)",2
"List(Charlie, Brown)",2
